In [339]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pymorphy2
nltk.download('punkt')
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/operator3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/operator3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [476]:
df = pd.read_excel('profsa/Реестр 327 тыс. деклараций ЕП РФ без 140000-200000.xlsx', sheet_name='все ДС с кодами')

In [477]:
df_category = pd.read_excel('profsa/Реестр 327 тыс. деклараций ЕП РФ без 140000-200000.xlsx', sheet_name='Список разделов из ЕП РФ (коды)')

In [478]:
df_category

,Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции),Подкатегория продукции,Количество ДС по коду
0,9300.10,"Лекарственные средства, зарегистрированные в у...",207841
1,2364.10,Смеси строительные,12107
2,3482.20,Аккумуляторы и аккумуляторные батареи никель-м...,5059
3,5990.10,"Посуда керамическая (фарфоровая, полуфарфорова...",4152
4,5970.10,Посуда из стекла для взрослых,3999
...,...,...,...
398,9386.00,Бактериофаги (включая бактериофаги для ветерин...,1
399,5460.00,"Тетради школьные ученические, обои и товары бу...",1
400,3185.36,Статические преобразователи для устройств элек...,1
401,3457.00,"Аппаратура, специально спроектированная для эл...",1


In [340]:
all_sents = []
for index, row in df.iterrows():
    all_sents.append(row['Общее наименование продукции'])
    all_sents.append(row['Подкатегория продукции'])

In [341]:
categories = []
for index, row in df_category.iterrows():
    categories.append(row['Подкатегория продукции'])

In [6]:
morph = pymorphy2.MorphAnalyzer()

In [342]:
def make_sent_good(sent):
    
    #print(sent)
    tokens = word_tokenize(sent, language="russian")
    #print(tokens)
    filtered_tokens = []
    stop_words = stopwords.words("russian")
    for token in tokens:
        if token not in stop_words and token.isalpha():
#            parsed = morph.parse(token)
#            filtered_tokens.append(parsed[0].normal_form)
            filtered_tokens.append(token.lower())
    return filtered_tokens

In [343]:
make_sent_good(all_sents[0])


['посуда', 'керамическая', 'взрослых', 'кружки', 'логотипом', 'lipton']

In [344]:
dataset = [make_sent_good(sent) for sent in all_sents ]

In [345]:
categories = [make_sent_good(sent) for sent in categories]

In [14]:
import pickle


In [15]:
with open('posuda_tokenized.pickle', 'wb') as f:
    pickle.dump(dataset, f)

In [96]:
with open('categories_tokenized.pickle', 'wb') as f:
    pickle.dump(categories, f)

In [105]:
len(categories)

14

In [364]:
w2v_model = Word2Vec(dataset, min_count=1)

In [365]:
#https://medium.com/@bigdataschool/%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5-nlp-%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8-word2ve%D1%81-%D0%BD%D0%B0-%D1%80%D1%83%D1%81%D1%81%D0%BA%D0%B8%D1%85-%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0%D1%85-%D1%81-python-aa10528b99c1
w2v_model.train(dataset,total_examples=w2v_model.corpus_count, epochs=500)

(123213875, 177715000)

In [366]:
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary


dct = Dictionary(dataset)  # fit dictionary
corpus = [dct.doc2bow(line) for line in dataset]  # convert corpus to BoW format
tfidf_model = TfidfModel(corpus)  # fit model

In [67]:
tfidf_model[dct.doc2bow(['алюминевый','посуда'])]

[(5, 0.010872943835771207), (1391, 0.999940887799046)]

In [367]:
dct.doc2bow(['алюминевый','посуда'])

[(5, 1), (1815, 1)]

In [73]:
w2v_model.wv.get_vector('алюмин').shape

(100,)

In [76]:
type(w2v_model.wv.get_vector('алюминь'))

KeyError: "Key 'алюминь' not present"

In [75]:
morph.parse('алюминь')[0].normal_form

'алюминь'

In [429]:
def get_sent_vector(sent):
    return d2v_model.infer_vector(sent)
    words_vec = [w2v_model.wv.get_vector(word) for word in sent]
    res = np.zeros(words_vec[0].shape)
    tf_idf_res = tfidf_model[dct.doc2bow(sent)]
    for word_vec,  tfidf_coef in zip(words_vec, tf_idf_res):

        res += tfidf_coef[1] * word_vec
    res /= words_vec[0].shape[0]
    return res

In [397]:
get_sent_vector(dataset[0])

array([ 0.02781032,  0.4085047 , -0.44702992,  0.0539773 , -0.04310822,
        0.17545351, -0.13320844, -0.3056218 ,  0.34841716,  0.19193138,
        0.15564863, -0.06491449,  0.04263204, -0.20311144, -0.01146553,
       -0.11866992,  0.01041583, -0.2678282 ,  0.1257502 ,  0.15241073,
       -0.04545007,  0.24235094, -0.08934365,  0.19774266,  0.02240207,
        0.04041701,  0.11465981, -0.15462114,  0.01058418,  0.00385917,
       -0.22416271, -0.04272363, -0.311184  ,  0.18536225, -0.13066389,
        0.0860125 ,  0.02188387, -0.09474594,  0.11391672, -0.1921134 ,
        0.09151542, -0.2922439 , -0.25543404,  0.25732645,  0.27427024,
        0.1620725 , -0.10673653,  0.01912489,  0.38331243,  0.02361956,
        0.44966185, -0.39357105, -0.15504345, -0.21297616, -0.14395915,
        0.0376996 , -0.16237636, -0.14683363, -0.15797293, -0.10006488,
        0.2838262 ,  0.11812484,  0.07483999,  0.0791963 , -0.15667215,
       -0.3462194 , -0.08015029, -0.19231793,  0.04470037, -0.10

In [472]:
categories_vec = [get_sent_vector(sent) for sent in categories]

In [349]:
len(categories_vec)

14

In [431]:
def get_best(sent):
    vec = get_sent_vector(sent)
    similarity = []
    for i in range(len(categories)):
        dot = np.dot(vec, categories_vec[i])
        norm_u = np.sqrt(np.sum(vec**2))
        norm_v = np.sqrt(np.sum(categories_vec[i]**2))
        cosine_similarity = dot / (norm_u* norm_v)
        similarity.append(abs(cosine_similarity))
    return similarity


In [432]:
def get_best2(sent):
    vec = get_sent_vector(sent)
    similarity = []
    for i in range(len(categories)):
        u = vec / np.sum(vec)
        v = categories_vec[i] / np.sum(categories_vec[i])
        cosine_similarity = np.sum((v - u)**2)
        similarity.append(cosine_similarity)
    return similarity

In [475]:

i = 2
res = np.array(get_best(dataset[i]))
i_min = res.argmin()
print(res)
print(i_min)
print(dataset[i])
print(categories[i_min])
print(1 - res / np.sum(res))
print(res[i_min])



[0.65298    0.7470947  0.6189548  0.62036055 0.64599264 0.6603598
 0.5746767  0.6406572  0.34420848 0.6673304  0.3918433  0.5928627
 0.64368796 0.5425581 ]
8
['посуда', 'взрослых', 'коррозионностойкой', 'стали']
['посуда', 'керамическая', 'фарфоровая', 'полуфарфоровая', 'фаянсовая', 'майоликовая', 'взрослых']
[0.9217385  0.91045856 0.92581654 0.92564803 0.92257595 0.92085403
 0.9311234  0.92321545 0.95874566 0.92001855 0.9530365  0.92894375
 0.92285216 0.9349729 ]
0.34420848


In [229]:
dataset[0]

['Посуда', 'керамическая', 'взрослых', 'кружки', 'логотипом', 'Lipton']

In [194]:
dataset[1]

['Посуда',
 'керамическая',
 'фарфоровая',
 'полуфарфоровая',
 'фаянсовая',
 'майоликовая',
 'взрослых']

In [211]:
get_best(dataset[2])

[0.4063737,
 0.40637377,
 0.2371112,
 0.2371112,
 0.15085855,
 0.40637377,
 0.32153106,
 0.4217005,
 0.4063737,
 0.09332343,
 0.23711118,
 0.15085855,
 0.35708866,
 0.4678715]

In [178]:
all_sents[0]

'Посуда керамическая для взрослых:кружки с логотипом "Lipton"'

In [179]:
all_sents[1]

'Посуда керамическая (фарфоровая, полуфарфоровая, фаянсовая, майоликовая) для взрослых'

In [180]:
dataset[2]

['Посуда', 'взрослых', 'коррозионностойкой', 'стали']

In [227]:
categories[12]

['Посуда', 'чугунная', 'черная']

In [214]:
dataset[2]

['Посуда', 'взрослых', 'коррозионностойкой', 'стали']

In [322]:
import fasttext

In [324]:
ft = fasttext.load_model('cc.ru.300.bin')

In [325]:
ft.get_dimension()

300

In [331]:
ft.get_word_vector(dataset[0][0])

array([ 0.02499536, -0.02028517, -0.09813499, -0.10727128,  0.10574081,
        0.0161325 ,  0.10150688,  0.02346335, -0.00836308, -0.04549556,
       -0.08452549,  0.10351308, -0.00691217, -0.07391488, -0.05252884,
        0.14316516,  0.09065375, -0.06574064,  0.04293767,  0.02332023,
        0.00568039, -0.02961118, -0.00391035,  0.08638075, -0.08155018,
        0.07115488, -0.06242096, -0.02066853,  0.03229598,  0.05967253,
        0.03437259, -0.10345087, -0.08954046, -0.06979303, -0.05757707,
       -0.01896729, -0.06720374, -0.1167922 ,  0.03613598, -0.05500325,
        0.06694788,  0.07080831, -0.0037757 ,  0.026268  , -0.00737507,
        0.06156323,  0.05973203, -0.10568403,  0.08991084,  0.03073901,
        0.03606437, -0.0369144 ,  0.04332462,  0.05498862, -0.04544231,
       -0.08502398, -0.01505822, -0.03462733, -0.00989054,  0.06426308,
       -0.03130373,  0.03298218,  0.03366033,  0.00819515,  0.01649939,
       -0.02235121,  0.00877583,  0.0120823 , -0.01483226, -0.07

In [380]:
from gensim.models import Doc2Vec

In [381]:
d2v_model  = Doc2Vec(dataset)

AttributeError: 'list' object has no attribute 'words'

In [383]:
documents = []
for i, w in enumerate(dataset):
    documents.append(gensim.models.doc2vec.TaggedDocument(w, [i]))

TaggedDocument(words=['посуда', 'керамическая', 'взрослых', 'кружки', 'логотипом', 'lipton'], tags=[0])

In [469]:
d2v_model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=1, dm=1)

In [470]:
d2v_model.build_vocab(documents)

In [471]:
max_epochs = 20
vec_size = 20
alpha = 0.025

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    d2v_model.train(documents,
                total_examples=d2v_model.corpus_count,
                epochs=d2v_model.epochs)
    # decrease the learning rate
    d2v_model.alpha -= 0.0002
    # fix the learning rate, no decay
    d2v_model.min_alpha = d2v_model.alpha

#model.save("d2v.model")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
Model Saved


In [450]:
d2v_model.train(documents)

ValueError: You must specify either total_examples or total_words, for proper learning-rate and progress calculations. If you've just built the vocabulary using the same corpus, using the count cached in the model is sufficient: total_examples=model.corpus_count.

In [448]:
d2v_model.wv.get_vector(documents)

TypeError: unhashable type: 'list'

In [445]:
from gensim.test.utils import get_tmpfile

#fname = get_tmpfile("doc2vec_model.pt")

d2v_model.save("doc2vec_model.pt")

In [449]:
d2v_model.infer_vector(dataset[0])

array([ 4.9741031e-04, -1.1396804e-03, -3.6502708e-04,  2.4061282e-04,
        1.5370694e-03, -1.3068444e-03, -9.9943276e-04,  2.6318829e-05,
        2.0669123e-04, -4.2814514e-04,  1.7092109e-04,  8.0774346e-04,
        1.9846737e-04, -1.1614856e-03, -1.3855828e-03, -5.3433614e-04,
        7.5544615e-04,  1.7852009e-04,  9.6866809e-04,  1.6451827e-03,
        1.2908826e-03,  7.0024031e-04,  4.1137714e-04, -9.1026642e-04,
        3.1481424e-04, -1.1642694e-03, -2.7795980e-04, -1.5088087e-03,
        3.2496054e-04, -1.1019765e-03,  1.2382535e-03,  9.9714741e-04,
       -5.1405816e-04,  1.1994574e-03,  6.5655174e-04, -7.0197490e-04,
       -1.2724707e-04,  1.4914417e-03,  2.6703338e-04, -1.0180123e-03,
        1.5447185e-03,  2.6629030e-04,  1.1072060e-04,  3.2431126e-04,
       -2.0329704e-04, -1.0982716e-03,  8.5493329e-04, -6.3437881e-04,
        1.0087359e-03,  2.3080091e-04, -7.3725689e-04, -6.4860506e-04,
       -1.6491911e-03,  3.9974053e-05,  1.1195479e-03, -8.2726509e-04,
      